In [13]:
! pip install --upgrade pip
! pip install openai
! pip install langchain
! pip install langchain_openai
! pip install openpyxl
! pip install pandas

  Using cached tiktoken-0.5.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached regex-2023.12.25-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
Using cached tiktoken-0.5.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
Using cached regex-2023.12.25-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (789 kB)


In [24]:
import openpyxl
import openai
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import pandas as pd
import json
from langchain_core.utils.function_calling import convert_to_openai_tool

In [7]:
wb = openpyxl.load_workbook(filename='data/sample_input.xlsx')

In [6]:
wb.sheetnames

["NOW-US, IS FY'23", "NOW-US, BS FY'23", "NOW-US, CF FY'23"]

In [8]:
wb.active

<Worksheet "NOW-US, CF FY'23">

In [9]:
ws = wb.active

In [10]:
ws.max_column, ws.max_row

(4, 61)

In [16]:
df = pd.read_excel('data/sample_input.xlsx')
df

,FactSet Filings Wizard,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,SERVICENOW INC,NaN,NaN,NaN
1,NOW-US,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,Annual Report - FY 2023 - Income Statement,NaN,NaN,NaN
4,Open in Workstation,NaN,NaN,NaN
5,Open in Web,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,Consolidated Statements of Comprehensive Incom...,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN
9,NaN,"12 Months Ended Dec. 31, 2023","12 Months Ended Dec. 31, 2022","12 Months Ended Dec. 31, 2021"


In [17]:
str(df)

'                               FactSet Filings Wizard  \\\n0                                      SERVICENOW INC   \n1                                              NOW-US   \n2                                                 NaN   \n3          Annual Report - FY 2023 - Income Statement   \n4                                 Open in Workstation   \n5                                         Open in Web   \n6                                                 NaN   \n7   Consolidated Statements of Comprehensive Incom...   \n8                                                 NaN   \n9                                                 NaN   \n10                                                NaN   \n11                                          Revenues:   \n12                                     Total revenues   \n13                                                NaN   \n14                                  Cost of revenues:   \n15                         Total cost of revenues [1]   \n16          

In [27]:
def highlight_cell(row: str, col: str):
    """Highlight the given cell as containing data of interest.

    Args:
        row: label of the row of the cell
        col: label of the column of the cell
    """
    pass

highlight = convert_to_openai_tool(highlight_cell)

In [34]:
llm = ChatOpenAI(model_name='gpt-4').bind_tools([highlight_cell])
system_prompt = 'You are a financial advisor helping users find information in tables. You will be given a financial statement table and a particular piece of information to find in that table. The table might be formatted in different ways, but there will likely be labels for kinds of data on a header row and column, and corresponding data points further on that row or column. Your task is to highlight all cells that contain the kind of data that the user is looking for.'
human_template = "I'm looking for {data_title} in this table:\n{table_string}"
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('human', human_template),
])
chain = prompt | llm

In [35]:
table_string = str(df)
res = chain.invoke({'data_title': 'Cost of Goods Sold (COGS)', 'table_string': table_string})

In [36]:
res.additional_kwargs

{'tool_calls': [{'id': 'call_mzdfllflFCItJsDZZXP1J0lJ',
   'function': {'arguments': '{\n"row": "Total cost of revenues [1]",\n"col": "12 Months Ended Dec. 31, 2023"\n}',
    'name': 'highlight_cell'},
   'type': 'function'}]}

In [22]:
res2 = chain.invoke({'data_title': 'Revenue', 'table_string': table_string})

In [23]:
res2.content

'The "Revenue" data is located at the following coordinates:\n\n- Total revenues: (12, "Unnamed: 1"), (12, "Unnamed: 2"), (12, "Unnamed: 3")\n- Total revenues: (41, "Unnamed: 1"), (41, "Unnamed: 2"), (41, "Unnamed: 3")\n- Total revenues: (48, "Unnamed: 1"), (48, "Unnamed: 2"), (48, "Unnamed: 3")'

{
  "type": "function",
  "function": {
    "name": "highlight_cell",
    "description": "Highlight the given cell as containing data of interest.",
    "parameters": {
      "type": "object",
      "properties": {
        "row": {
          "type": "string",
          "description": "label of the row of the cell"
        },
        "col": {
          "type": "string",
          "description": "label of the column of the cell"
        }
      },
      "required": [
        "row",
        "col"
      ]
    }
  }
}
